# NNLM Neural Network Language Model

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim

In [48]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [49]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [50]:
class NNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, context_size):
        super(NNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.fc1 = nn.Linear(context_size * embed_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
    def forward(self, x):
        embeds = self.embed(x)                       # (batch_size, context_size, embed_size)
        embeds = embeds.view(embeds.size(0), - 1)    # (batch_size, context_size * embed_size)
        output = self.fc1(embeds)
        output = self.fc2(output)
        log_probs = self.log_softmax(output)
        return log_probs

In [51]:
VOCAB_SIZE = 5000
EMBED_SIZE = 300
HIDDEN_SIZE = 128
CONTEXT_SIZE = 2

model = NNLM(VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, CONTEXT_SIZE)
print(model)

NNLM(
  (embed): Embedding(5000, 300)
  (fc1): Linear(in_features=600, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=5000, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)


In [52]:
X = torch.randint(0, VOCAB_SIZE, (8, CONTEXT_SIZE)) # 0 ~ VOCAB_SIZE인 랜덤정수를 (8, CONTEXT_SIZE) 크기로 
y = torch.randint(0, VOCAB_SIZE, (8,))

In [53]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
optimizer.zero_grad()
output = model(X)
loss = criterion(output, y)
loss.backward()
optimizer.step()

print(loss.item())

8.46241283416748
